<a href="https://colab.research.google.com/github/complyservice/modelowebscapingdados-dosestadosbrasileiros/blob/main/Webscraping_modelo_IBGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd 

In [2]:
def scrap_state_info(state: str) -> dict:
    """
    Retorna informações do estado brasileiro

    :param state: nome do estado
    :returns state_dict: dicionário com indicadores do estado
    """
    print(f'⛷️ Picking {state} info...')
    state_url = f'https://www.ibge.gov.br/cidades-e-estados/{state}.html'
    page = requests.get(state_url)

    soup = BeautifulSoup(page.content, 'html.parser')
    indicadors = soup.select('.indicador')

    state_dict = {
        ind.select('.ind-label')[0].text: ind.select('.ind-value')[0].text
        for ind in indicadors
    }

    state_dict['Estado'] = state

    return state_dict

scrap_state_info('sp')

⛷️ Picking sp info...


{'Capital': 'São Paulo\xa0\xa0\xa0[2010]',
 'Densidade demográfica': '166,25 hab/km²\xa0\xa0\xa0[2010]',
 'Despesas empenhadas': '231.982.243,69 R$ (×1000)\xa0\xa0\xa0[2017]',
 'Estado': 'sp',
 'Gentílico': 'paulista',
 'Governador': 'JOÃO AGRIPINO DA COSTA DORIA JUNIOR\xa0\xa0\xa0[2019]',
 'IDH Índice de desenvolvimento humano': '0,783\xa0\xa0\xa0[2010]',
 'Matrículas no ensino fundamental': '5.414.208 matrículas\xa0\xa0\xa0[2020]',
 'População estimada': '46.649.132 pessoas\xa0\xa0\xa0[2021]',
 'Receitas realizadas': '232.822.496,57 R$ (×1000)\xa0\xa0\xa0[2017]',
 'Rendimento mensal domiciliar per capita': '1.814 R$\xa0\xa0\xa0[2020]',
 'Total de veículos': '30.778.960 veículos\xa0\xa0\xa0[2020]',
 'Área Territorial': '248.219,481 km²\xa0\xa0\xa0[2020]'}

In [ ]:
#Passa pelas paginas de cada estado
states = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO']

states_data = [scrap_state_info(state) for state in states]

⛷️ Picking AC info...
⛷️ Picking AL info...
⛷️ Picking AP info...
⛷️ Picking AM info...
⛷️ Picking BA info...
⛷️ Picking CE info...
⛷️ Picking DF info...
⛷️ Picking ES info...
⛷️ Picking GO info...
⛷️ Picking MA info...
⛷️ Picking MT info...
⛷️ Picking MS info...
⛷️ Picking MG info...
⛷️ Picking PA info...
⛷️ Picking PB info...
⛷️ Picking PR info...
⛷️ Picking PE info...
⛷️ Picking PI info...
⛷️ Picking RJ info...
⛷️ Picking RN info...
⛷️ Picking RS info...
⛷️ Picking RO info...
⛷️ Picking RR info...
⛷️ Picking SC info...
⛷️ Picking SP info...
⛷️ Picking SE info...
⛷️ Picking TO info...


In [ ]:
df = pd.DataFrame(states_data)
df.head()
#df.info()

,Governador,Capital,Gentílico,Área Territorial,População estimada,Densidade demográfica,Matrículas no ensino fundamental,IDH Índice de desenvolvimento humano,Receitas realizadas,Despesas empenhadas,Rendimento mensal domiciliar per capita,Total de veículos,Estado
0,GLADSON DE LIMA CAMELI [2019],Rio Branco [2010],acriano,"164.173,431 km² [2020]",894.470 pessoas [2020],"4,47 hab/km² [2010]",156.679 matrículas [2020],"0,663 [2010]","6.632.883,11 R$ (×1000) [2017]","6.084.416,81 R$ (×1000) [2017]",917 R$ [2020],306.258 veículos [2020],AC
1,JOSE RENAN VASCONCELOS CALHEIROS FILHO [2019],Maceió [2010],alagoano,"27.830,656 km² [2020]",3.351.543 pessoas [2020],"112,33 hab/km² [2010]",464.704 matrículas [2020],"0,631 [2010]","11.950.438,46 R$ (×1000) [2017]","10.460.634,92 R$ (×1000) [2017]",796 R$ [2020],930.933 veículos [2020],AL
2,ANTONIO WALDEZ GÓES DA SILVA [2019],Macapá [2010],amapaense,"142.470,762 km² [2020]",861.773 pessoas [2020],"4,69 hab/km² [2010]",134.820 matrículas [2020],"0,708 [2010]","5.396.417,14 R$ (×1000) [2017]","4.224.464,09 R$ (×1000) [2017]",893 R$ [2020],215.330 veículos [2020],AP
3,WILSON MIRANDA LIMA [2019],Manaus [2010],amazonense,"1.559.167,878 km² [2020]",4.207.714 pessoas [2020],"2,23 hab/km² [2010]",700.104 matrículas [2020],"0,674 [2010]","17.328.459,43 R$ (×1000) [2017]","15.324.896,56 R$ (×1000) [2017]",852 R$ [2020],969.722 veículos [2020],AM
4,RUI COSTA DOS SANTOS [2019],Salvador [2010],baiano,"564.760,427 km² [2020]",14.930.634 pessoas [2020],"24,82 hab/km² [2010]",1.947.177 matrículas [2020],"0,660 [2010]","50.191.003,24 R$ (×1000) [2017]","45.570.160,00 R$ (×1000) [2017]",965 R$ [2020],4.506.825 veículos [2020],BA


In [ ]:
#Limpeza dos dados
states_df = df.copy()
states_df.columns = ['governor', 'capital', 'gentile', 'area', 'population', 'demographic_density', 'primary_school_enrollment', 'idh', 'realized_revenue', 'committed_expenses', 'income_per_capita', 'vehicle_total', 'code']
states_df = states_df[['code', 'governor', 'population', 'area', 'idh', 'income_per_capita', 'vehicle_total', 'primary_school_enrollment', 'committed_expenses', 'realized_revenue']]


states_df = states_df.replace({
    '\.': '',
    ',': '.',
    '\[\d+\]': '',
    ' hab/km²': '',
    ' km²': '',
    ' pessoas': '',
    ' matrículas': '',
    'R\$.*': '',
    ' veículos': ''
}, regex=True)

states_df.head()

num_cols = ['population', 'area', 'idh', 'income_per_capita', 'vehicle_total', 'primary_school_enrollment', 'committed_expenses', 'realized_revenue']

states_df[num_cols] = states_df[num_cols].apply(lambda x: x.str.strip())

In [ ]:
#Exportar para o Drive

from google.colab import drive
drive.mount('/content/drive')

states_df.to_csv('/content/drive/MyDrive/SimpleData.ai/Scrap/Data/ibge_states.csv')